In [2]:
from pyparsing import Word, alphas, nums, Literal, Or, ParseException, ZeroOrMore, Group, delimitedList, OneOrMore
from enum import Enum

identifier = Word(alphas)
value = Word(nums)
variable = identifier ^ value

clause = identifier.setResultsName("predicate") + "(" + delimitedList(variable, delim=",").setResultsName("variables") + ")"
rule = clause.setResultsName("head") + ":-" + delimitedList(Group(clause), delim=",").setResultsName("body")
statement = Group((clause ^ rule) + Or([".", "?"]))
program = OneOrMore(statement)

In [133]:
from collections import defaultdict
from copy import deepcopy

def is_variable(v):
    return v[0].isalpha() and v[0].isupper()

def find_matching_indices(l1, l2):
    r = list()
    for el in l2:
        if el in l1:
            r.append(l1.index(el))
        else:
            r.append(None)
    return r

def handle(memory, s):
    new_statements = list()
    if "head" in s:
        locs = variable_locations(s)
        if "assignments" not in s:
            s["assignments"] = defaultdict(lambda:None)
        if len(s["body"]) == 0:
            memory[s["head"]["predicate"]].append(s["head"]["variables"])
        for cidx in range(len(s["body"])):
            c = s["body"][cidx]
            if any(is_variable(var) for var in c["variables"]):
                for values in memory[c["predicate"]]:
                    new_s = deepcopy(s)
#                     print("Before:", rule_to_string(new_s))
                    try:
                        substitutions = generate_substitutions(c, values)
                        new_s = make_substitutions(new_s, locs, substitutions)
                        new_s["body"] = new_s["body"][:cidx] + new_s["body"][cidx+1:]
#                         print("After:", rule_to_string(new_s))
                        new_statements.append(new_s)
                    except SubstitutionConflict as e:
#                         print(e)
                        pass
#                     print()
            else:
                # just remove the already fulfilled clause if the clause matches some fact in memory
                if clause_is_valid(memory, c):
                    new_s = deepcopy(s)
                    new_s["body"] = new_s["body"][:cidx] + new_s["body"][cidx+1:]
                    new_statements.append(new_s)
    else:
        memory[s["predicate"]].append(s["variables"])
    return new_statements

def clause_is_valid(memory, clause):
    valid = False
    for values in memory[clause["predicate"]]:
        if all(values[i] == clause["variables"][i] for i in range(len(values))):
            valid = True
            break
    return valid

def clause_to_string(clause):
    return clause["predicate"] + "(" + ", ".join(clause["variables"]) + ")"

def rule_to_string(rule):
    return clause_to_string(rule["head"]) + " :- " + ", ".join(clause_to_string(bc) for bc in rule["body"])

class SubstitutionConflict(Exception):
    pass

def generate_substitutions(clause, new_values):
    candidates = dict(zip(clause["variables"], new_values))
#     print("Trying to make substitutions", ", ".join([var + " -> " + value for var, value in candidates.items()]))
    for vidx in range(len(clause["variables"])):
        var = clause["variables"][vidx]
        if not is_variable(var):
#             print("Found prior value", var, value)
            if var != new_values[vidx]:
                raise SubstitutionConflict("Conflicting values: " + var + value)
    return candidates

def make_substitutions(rule, locs, substitutions):
#     print("Making substitutions", ", ".join([var + " -> " + value for var, value in substitutions.items()]))
#     print(locs)
    for var, value in substitutions.items():
        if rule["assignments"][var] is None:
            for vidx in locs[var]["head"]:
                rule["head"]["variables"][vidx] = value
            for cidx in range(len(locs[var]["body"])):
                c = locs[var]["body"][cidx]
                for vidx in c:
                    rule["body"][cidx]["variables"][vidx] = value
        else:
#             print("Found prior value", rule["assignments"][var], value)
            if rule["assignments"][var] != value:
                raise SubstitutionConflict("Conflicting values: " + rule["assignments"][var] + value)
    return rule

def variable_locations(rule):
    num_clauses = len(rule["body"])
    locs = defaultdict(lambda:{"head":list(), "body":[[] for i in range(num_clauses)]})
    for cidx in range(len(rule["body"])):
        clocs = defaultdict(list)
        for vidx in range(len(rule["body"][cidx]["variables"])):
            varname = rule["body"][cidx]["variables"][vidx]
            clocs[varname].append(vidx)
        for varname, l in clocs.items():
            locs[varname]["body"][cidx] = l
    for vidx in range(len(rule["head"]["variables"])):
        varname = rule["head"]["variables"][vidx]
        locs[varname]["head"].append(vidx)
    return locs
            
def evaluate(statements):
    statements = [s.asDict() for s in statements]
    memory = defaultdict(list)
    while len(statements) > 0:
        new_statements = handle(memory, statements[0])
        statements = statements[1:] + new_statements
#         print("Memory:", memory)
    return memory


In [134]:
evaluate(program.parseString("""
parent(alice, eve).
parent(bob, eve).
parent(zach, alice).
parent(yolanda, bob).
sibling(A,B) :- parent(A,C),parent(B,C).
cousin(Z,Y) :- parent(Z,A),parent(Y,B),sibling(A,B).
"""))["cousin"]

[['zach', 'zach'],
 ['zach', 'yolanda'],
 ['yolanda', 'zach'],
 ['yolanda', 'yolanda'],
 ['zach', 'zach'],
 ['yolanda', 'zach'],
 ['zach', 'yolanda'],
 ['yolanda', 'yolanda']]

In [135]:
evaluate(program.parseString("""
edge(1,2).
edge(2,3).
tc(A,B) :- edge(A,B).
tc(A,B) :- edge(A,C),tc(C,B).
"""))["tc"]

[['1', '2'], ['2', '3'], ['1', '3']]

In [21]:
variable_locations({'predicate': 'sibling', 'variables': ['A', 'B'], 'head': {'predicate': 'sibling', 'variables': ['A', 'B']}, 'body': [{'predicate': 'parent', 'variables': ['A', 'C']}, {'predicate': 'parent', 'variables': ['B', 'C']}]})

{'predicate': 'sibling', 'variables': ['A', 'B'], 'head': {'predicate': 'sibling', 'variables': ['A', 'B']}, 'body': [{'predicate': 'parent', 'variables': ['A', 'C']}, {'predicate': 'parent', 'variables': ['B', 'C']}]}
{'predicate': 'sibling', 'variables': ['A', 'B'], 'head': {'predicate': 'sibling', 'variables': ['A', 'B']}, 'body': [{'predicate': 'parent', 'variables': ['A', 'C']}, {'predicate': 'parent', 'variables': ['B', 'C']}]}
{'predicate': 'sibling', 'variables': ['A', 'B'], 'head': {'predicate': 'sibling', 'variables': ['A', 'B']}, 'body': [{'predicate': 'parent', 'variables': ['A', 'C']}, {'predicate': 'parent', 'variables': ['B', 'C']}]}
{'predicate': 'sibling', 'variables': ['A', 'B'], 'head': {'predicate': 'sibling', 'variables': ['A', 'B']}, 'body': [{'predicate': 'parent', 'variables': ['A', 'C']}, {'predicate': 'parent', 'variables': ['B', 'C']}]}


defaultdict(<function __main__.variable_locations.<locals>.<lambda>>,
            {'A': {'body': [[0]], 'head': [0]},
             'B': {'body': [[0]], 'head': [1]},
             'C': {'body': [[1], [1]], 'head': []}})

In [67]:
a = defaultdict(list)
a["foo"].append(0)
b = deepcopy(a)
b["foo"][0] = 1
print(a,b)

defaultdict(<class 'list'>, {'foo': [0]}) defaultdict(<class 'list'>, {'foo': [1]})


In [ ]:
"""
person(alice).
person(bob).
person(eve).
person(zach).
person(yolanda).
"""